In [1]:
import random
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
from transformers import AutoTokenizer
from tqdm import tqdm
import scml
from scml import nlp as snlp
import mylib

In [2]:
percentiles = [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [3]:
pretrained_dir = "pretrained/google/electra-small-discriminator"
model_max_length = 512
tokenizer = AutoTokenizer.from_pretrained(pretrained_dir, model_max_length=model_max_length)
print(f"{repr(tokenizer)}\n{tokenizer.model_input_names}")

PreTrainedTokenizerFast(name_or_path='pretrained/google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
['input_ids', 'token_type_ids', 'attention_mask']


In [4]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661 entries, 0 to 19660
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             19661 non-null  object
 1   pub_title      19661 non-null  object
 2   dataset_title  19661 non-null  object
 3   dataset_label  19661 non-null  object
 4   cleaned_label  19661 non-null  object
dtypes: object(5)
memory usage: 768.1+ KB


In [5]:
gt_map: Dict[str, List[str]] = {}
dl_map: Dict[str, List[str]] = {}
for t in tqdm(train.itertuples()):
    pid = getattr(t, "Id")
    cl = getattr(t, "cleaned_label")
    dl = getattr(t, "dataset_label")
    if pid not in gt_map:
        gt_map[pid] = list()
    if pid not in dl_map:
        dl_map[pid] = list()
    gt_map[pid].append(cl)
    dl_map[pid].append(dl)

19661it [00:00, 154887.09it/s]


In [6]:
rows = []
sep = "|"
m = 900
max_negative_examples = 10
max_length = 502
step = 100
for rid, gts in tqdm(gt_map.items()):
    is_multi = 0
    if len(gts) > 1:
        is_multi = 1
    ground_truth = sep.join(sorted(gts))
    dataset_labels = sep.join(sorted(dl_map[rid]))
    with open(f"input/train/{rid}.json") as in_file:
        sections = json.load(in_file)
    tmp = []
    for section in sections:
        tmp.append(section["text"])
    text = " ".join(tmp).strip()
    text = snlp.to_ascii_str(text)
    spans = []
    for dl in dl_map[rid]:
        i = 0
        j = len(text)
        while i < j - len(dl) + 1:
            p = text.find(dl, i, j)
            if p == -1:
                break
            q = p + len(dl)
            spans.append((p, q))
            i = q
            # Right context: answer is on the left edge of window
            a = p
            b = 2 * m + a
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                b -= step * 2
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_start"] = 0
            row["answer_end"] = len(dl)
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)
            # Left context: answer is on the right edge of window
            b = q
            a = max(0, b - (2 * m))
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                a += step * 2
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["answer_end"] = b - a
            row["answer_start"] = row["answer_end"] - len(dl) 
            row["context"] = text[a:b]
            row["context_token_length"] = _len
            rows.append(row)
            # Left and right context: answer is in the middle of window
            a = max(0, p - m)
            b = q + m
            _len = len(tokenizer(text[a:b])["input_ids"])
            while _len > max_length and a < b:
                a += step
                b -= step
                _len = len(tokenizer(text[a:b])["input_ids"])
            row = {}
            row["Id"] = rid
            row["is_multi"] = is_multi
            row["ground_truth"] = ground_truth
            row["dataset_labels"] = dataset_labels
            row["is_impossible"] = 0
            row["context"] = text[a:b]
            row["answer_start"] = row["context"].index(dl)
            row["answer_end"] = row["answer_start"] + len(dl)
            row["context_token_length"] = _len
            rows.append(row)
    i = 0
    negative_examples = int(len(text) / (2 * m)) - len(spans)
    negative_examples = min(max_negative_examples, negative_examples)
    while i < negative_examples:
        b = len(text) - 1 - (2 * m)
        a = random.randint(0, b)
        b = 2 * m + a
        is_overlap = False
        for span in spans:
            if a <= span[0] <= b or a <= span[1] <= b:
                is_overlap = True
                break
        if is_overlap:
            continue
        _len = len(tokenizer(text[a:b])["input_ids"])
        while _len > max_length and a < b:
            a += step
            b -= step
            _len = len(tokenizer(text[a:b])["input_ids"])
        row = {}
        row["Id"] = rid
        row["is_multi"] = is_multi
        row["ground_truth"] = ground_truth
        row["dataset_labels"] = dataset_labels
        row["is_impossible"] = 1
        row["context"] = text[a:b]
        row["answer_start"] = -1
        row["answer_end"] = -1
        row["context_token_length"] = _len
        rows.append(row)
        i += 1

100%|██████████| 14316/14316 [05:12<00:00, 45.83it/s]


In [7]:
df = pd.DataFrame.from_records(rows)
cols = ["is_multi", "is_impossible"]
df[cols] = df[cols].astype(np.int8)
cols = ["answer_start", "answer_end", "context_token_length"]
df[cols] = df[cols].astype(np.int16) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302398 entries, 0 to 302397
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Id                    302398 non-null  object
 1   is_multi              302398 non-null  int8  
 2   ground_truth          302398 non-null  object
 3   dataset_labels        302398 non-null  object
 4   is_impossible         302398 non-null  int8  
 5   answer_start          302398 non-null  int16 
 6   answer_end            302398 non-null  int16 
 7   context               302398 non-null  object
 8   context_token_length  302398 non-null  int16 
dtypes: int16(3), int8(2), object(4)
memory usage: 11.5+ MB


In [8]:
df["is_multi"].value_counts(normalize=True)

1    0.516892
0    0.483108
Name: is_multi, dtype: float64

In [9]:
df["is_impossible"].value_counts(normalize=True)

0    0.605183
1    0.394817
Name: is_impossible, dtype: float64

In [10]:
assert (df["context_token_length"] <= max_length).all()
df["context_token_length"].describe(percentiles=percentiles)

count    302398.000000
mean        381.133083
std          57.314879
min           4.000000
1%          114.000000
5%          317.000000
10%         334.000000
20%         350.000000
30%         362.000000
40%         373.000000
50%         383.000000
60%         394.000000
70%         406.000000
80%         421.000000
90%         443.000000
95%         461.000000
99%         488.000000
max         502.000000
Name: context_token_length, dtype: float64

In [11]:
df.sample(20).head(20)

,Id,is_multi,ground_truth,dataset_labels,is_impossible,answer_start,answer_end,context,context_token_length
22944,7d060b54-4ccb-4a03-8b9e-6c76a78baac1,0,agricultural resource management survey,Agricultural Resource Management Survey,0,900,939,"d more than 70 percent of production on feeder-to-finish farms were under production contracts in 2004. Likewise, 67 percent of specialized farrowing operations and more than 90 percent of specialized weanling operations used contractual arrangements. In contrast, contract production was virtually nonexistent on farrow-to-finish farms.\nThe average size of hog operations increased from 1992 to 2004 but grew the fastest for operations producing under contract ( fig. 6 ). Contract feeder pig-to-finish operations averaged about 1,000 more head produced in 1992 than did other operations. By 1998, contract operations averaged 3,700 more head than other operations, and the difference reached 4,500 head in 2004. Size of hog-finishing operations by business arrangement Head of hogs sold/removed per farm Sources: USDA, ERS using data from USDA's 1992 Farm Costs and Returns Survey and USDA 's 1998 Agricultural Resource Management Survey. 1998 1992 Other operations 0 2,000 4,000 6,000\n8,000\nContract hog-finishing operations produced 7,000 head annually on average in 2004, compared with 2,500 head on other operations.\nAmong farms with hogs, the average value of farm production from hog enterprises increased from 46 to 71 percent during 1992-2004 (see table 1 ). That is, hogs generated 71 percent of the total value of farm production on these farms. The share of farm product value from hogs increased most rapidly, from 35 to 72 percent, on hog-finishing operations. Sources of hog feed also indicate increasing specialization in hog production. Grain produced on the same farm accounted for half of the feed consumed by hogs in 1992 but fell below 20 percent by 2004. Again, specialized hogfinishing operations accounted for the fastest change (from 45 to 15 percent). The resulting farms-with greater shares of production value",378
58589,b17853de-dd78-47ce-b152-72f47f97bfb7,1,adni|alzheimer s disease neuroimaging initiative adni,ADNI|Alzheimer's Disease Neuroimaging Initiative (ADNI),0,0,4,"ADNIGO/2 on each plate. Scaling was done by dividing the global average of each metabolite level by its average within the plate. These batch effect adjusted values are included in the Intermediate Data Level 2. Overall, this scaling factor was small (typically less than 10%), as can be seen by the raw reported NIST SRM-1950 values reported in Online-only Table 2 .\nThe next step of QC involved filtering based on quality metrics. We routinely applied filter criteria to each of the metabolites (based on the blinded ADNI 1 duplicates or ADNI 2/GO triplicates) to allow only the most robust analytes to be included in downstream analysis. Separately for each cohort, we used a coefficient of variation (CV) <30% across plates to filter out metabolites with limited variation and therefore statistical power for analysis. Next, we used an intraclass correlation coefficient (ICC) between the values for the blinded duplicate (or triplicate) analyes >0.6. Finally, analytes with >40% of measurements below the lower limit of detection (<LOD) were filtered out. This filtered data represents the Level 3 data matrix. This filtering reduced the total number of analytes reported from 20 analytes (Level 2) to 15 analytes (Level 3). The filter QC results are presented in detail in the Supplementary Table 1 .\nThe next step in data processing performs missing value replacement, by imputing any values reported as reported as '<LOD' were using LOD/2 value for each specific analyte. Additionally, we screened for outliers for removal prior to analysis. In ADNI 1, there were a total of 71 samples identified as outliers based on the following criteria: 69 samples identified as non-fasting, 2 samples lacking corresponding body mass index (BMI) va

In [12]:
df.to_parquet("output/train.parquet", index=False)